In [44]:
import pandas as pd

In [45]:
df_exam = pd.read_csv('examination_feature_cardio.csv')
df_exam = df_exam.drop(columns = ['id'])
df_obj = pd.read_csv('objective_feature_cardio.csv')
df_obj = df_obj.drop(columns = ['id'])
file_path3 = "Resources/subjective_feature_cardio.csv"



In [46]:
df_obj['age'] = df_obj['age'] / 365
df_obj['age'] = df_obj['age'].map("{:.1f}".format)
df_obj

,age,gender,height,weight,cardiovascular_disease
0,50.4,Female,168,62,No
1,55.4,Male,156,85,Yes
2,51.7,Male,165,64,Yes
3,48.3,Female,169,82,Yes
4,47.9,Male,156,56,No
...,...,...,...,...,...
69995,52.7,Female,168,76,No
69996,61.9,Male,158,126,Yes
69997,52.2,Female,183,105,Yes
69998,61.5,Male,163,72,Yes


In [47]:
# Removing  outliers in the 'weight' coulmn that less than 30 Kg
df_obj.drop(df_obj[df_obj['weight'] < 30].index, inplace = True)
# Removing  outliers in the 'height' coulmn that are less than  100
df_obj.drop(df_obj[df_obj['height'] < 100].index, inplace = True)
# Removing  outliers in the 'height' coulmn that are greater than  210
df_obj.drop(df_obj[df_obj['height'] > 210].index, inplace = True)

df_obj

,age,gender,height,weight,cardiovascular_disease
0,50.4,Female,168,62,No
1,55.4,Male,156,85,Yes
2,51.7,Male,165,64,Yes
3,48.3,Female,169,82,Yes
4,47.9,Male,156,56,No
...,...,...,...,...,...
69995,52.7,Female,168,76,No
69996,61.9,Male,158,126,Yes
69997,52.2,Female,183,105,Yes
69998,61.5,Male,163,72,Yes


In [48]:
# Remove outliers in the 'ap_hi' coulmn that are less than 60
df_exam.drop(df_exam[df_exam['ap_hi'] < 60].index, inplace = True)
# Remove outliers in the 'ap_lo' coulmn that are above 300
df_exam.drop(df_exam[df_exam['ap_hi'] > 300].index, inplace = True)

# Remove outliers in the 'ap_lo' coulmn that are below 10
df_exam.drop(df_exam[df_exam['ap_lo'] < 10].index, inplace = True)
# # Remove outliers in the 'ap_lo' coulmn that are greater than 100
df_exam.drop(df_exam[df_exam['ap_lo'] >= 100].index, inplace = True)

df_exam

,ap_hi,ap_lo,cholesterol,glucose
0,110,80,normal,normal
1,140,90,well above normal,normal
2,130,70,well above normal,normal
4,100,60,normal,normal
5,120,80,above normal,above normal
...,...,...,...,...
69995,120,80,normal,normal
69996,140,90,above normal,above normal
69997,180,90,well above normal,normal
69998,135,80,normal,well above normal


In [49]:
#create BMI datapoint
#BMI = (weight x weight)/(height)

df_obj["BMI"] = (df_obj['weight']*df_obj['weight'])/ df_obj['height']

df_obj['BMI'] = df_obj['BMI'].map("{:.0f}".format)
df_obj

,age,gender,height,weight,cardiovascular_disease,BMI
0,50.4,Female,168,62,No,23
1,55.4,Male,156,85,Yes,46
2,51.7,Male,165,64,Yes,25
3,48.3,Female,169,82,Yes,40
4,47.9,Male,156,56,No,20
...,...,...,...,...,...,...
69995,52.7,Female,168,76,No,34
69996,61.9,Male,158,126,Yes,100
69997,52.2,Female,183,105,Yes,60
69998,61.5,Male,163,72,Yes,32


In [51]:
df_obj.to_csv('cleaned_obj_data.csv')
df_exam.to_csv('cleaned_exam_data.csv')